In [ ]:
from datasets import load_dataset
from PIL import Image
import torch
import clip
import numpy as np
import json
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chargement du modèle et du processeur
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_model.eval()
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
# ComicsPAP Dataset HuggingFace
skill = "sequence_filling"
split = "val"
hf_dataset = load_dataset("VLR-CVC/ComicsPAP", skill, split=split, streaming=True)


results = []

README.md:   0%|          | 0.00/19.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/105 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/240 [00:00<?, ?it/s]

In [ ]:
for idx, sample in enumerate(tqdm(hf_dataset, desc="Processing CLIP embeddings")):
    # Traitement des images
    context_imgs = [img.convert("RGB") if isinstance(img, Image.Image) else Image.open(img).convert("RGB") for img in sample['context']]
    option_imgs = [img.convert("RGB") if isinstance(img, Image.Image) else Image.open(img).convert("RGB") for img in sample['options']]

    # Embeddings visuels (contexte)
    context_embs = []
    for img in context_imgs:
        with torch.no_grad():
            inputs = clip_processor(images=img, return_tensors="pt").to(device)
            emb = clip_model.get_image_features(**inputs).squeeze(0).cpu().numpy()
        context_embs.append(emb)
    context_img_avg = np.mean(context_embs, axis=0)

    # Embeddings visuels (options)
    option_embs = []
    for img in option_imgs:
        with torch.no_grad():
            inputs = clip_processor(images=img, return_tensors="pt").to(device)
            emb = clip_model.get_image_features(**inputs).squeeze(0).cpu().numpy()
        option_embs.append(emb)

    # Sauvegarde du résultat
    results.append({
        "sample_id": sample.get("sample_id", idx),
        "context_embedding": context_img_avg.tolist(),
        "option_embeddings": [e.tolist() for e in option_embs],
        "correct_index": sample["solution_index"],
        "missing_index": sample["index"]
    })



Processing CLIP embeddings: 262it [01:54,  2.29it/s]


In [ ]:
import torch

# შენახვა .pt ფაილში
torch.save(results, "comicspap_clip_embeddings.pt")
print("Saved CLIP embeddings to comicspap_clip_embeddings.pt")


Saved CLIP embeddings to comicspap_clip_embeddings.pt


In [ ]:
class ComicPAPDataset(Dataset):
   """

    Ce dataset charge les embeddings d'images de contexte et d'options,
    ainsi que les étiquettes associées (index correct et position manquante).

    """
   def __init__(self, path):
        self.samples = torch.load(path)

    def __len__(self):
      # Retourne le nombre total d'échantillons
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        context = torch.tensor(sample['context_embedding'], dtype=torch.float32)
        options = [torch.tensor(opt, dtype=torch.float32) for opt in sample['option_embeddings']]

        label = torch.tensor(sample['correct_index'], dtype=torch.long)
        pos_index = torch.tensor(sample.get('missing_index', sample.get('index', 0)), dtype=torch.long)

        # Regroupement des options en un seul tenseur [N, D]
        options = torch.stack(options)

        return {
            'context': context,          # [D]
            'options': options,          # [N, D]
            'label': label,              # scalar
            'position': pos_index        # scalar
        }


In [ ]:
def custom_collate_fn(batch):
    contexts = torch.stack([item['context'] for item in batch])                  # [B, 512]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)   # [B]
    positions = torch.tensor([item['position'] for item in batch], dtype=torch.long)  # [B]

    max_options = max(item['options'].size(0) for item in batch)

    padded_options = []
    for item in batch:
        opt = item['options']           # [N_i, 512]

        pad_len = max_options - opt.size(0)
        if pad_len > 0:
            pad = torch.zeros((pad_len, opt.size(1)), dtype=opt.dtype)
            opt = torch.cat([opt, pad], dim=0)

        padded_options.append(opt)           # [max_options, 512]

    return {
        'context': contexts,                        # [B, 512]
        'options': torch.stack(padded_options),     # [B, max_options, 512]
        'label': labels,                            # [B]
        'position': positions                       # [B]
    }


In [ ]:
import torch
import torch.nn as nn

class MLPPanelClassifier(nn.Module):
      """
    Classificateur de panneaux basé sur un perceptron multicouche (MLP).

    Le modèle projette les embeddings du contexte et des options dans un espace latent
    et concatène l'information de position avant de passer à travers un classificateur MLP.

    """
    def __init__(self, embedding_dim=512, position_dim=64, proj_dim=512):
        super().__init__()
        self.position_embedding = nn.Embedding(100, position_dim)

        # embeddings
        self.context_proj = nn.Linear(embedding_dim, proj_dim)
        self.option_proj = nn.Linear(embedding_dim, proj_dim)

        input_dim = proj_dim * 2 + position_dim

       # Classificateur MLP : 3 couches avec ReLU et Dropout
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

    def forward(self, context, options, position_idx):
        B, N, D = options.shape

        context = context.unsqueeze(1).expand(-1, N, -1)               # (B, N, 512)
        pos_emb = self.position_embedding(position_idx).unsqueeze(1).expand(-1, N, -1)  # (B, N, 64)


        context_proj = self.context_proj(context)              # (B, N, 512)
        option_proj = self.option_proj(options)                # (B, N, 512)

        x = torch.cat([context_proj, option_proj, pos_emb], dim=-1)    # (B, N, 1088)
        logits = self.classifier(x).squeeze(-1)                        # (B, N)
        return logits


In [ ]:
def train(model, dataloader, epochs=10, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            context = batch['context'].to(device)
            options = batch['options'].to(device)
            labels = batch['label'].to(device)
            position = batch['position'].to(device)


            logits = model(context, options, position)

            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        acc = correct / total * 100
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {acc:.2f}%")


In [ ]:

    dataset = ComicPAPDataset("comicspap_clip_embeddings.pt")
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

    # Model
    model = MLPPanelClassifier()

    # Training
    train(model, dataloader, epochs=50, lr=1e-4)



Epoch 1/50: 100%|██████████| 9/9 [00:00<00:00, 94.37it/s]


Epoch 1: Loss = 14.4857, Accuracy = 22.52%


Epoch 2/50: 100%|██████████| 9/9 [00:00<00:00, 90.85it/s]


Epoch 2: Loss = 14.4923, Accuracy = 21.37%


Epoch 3/50: 100%|██████████| 9/9 [00:00<00:00, 98.32it/s]


...

Epoch 48: Loss = 4.0820, Accuracy = 83.97%


Epoch 49/50: 100%|██████████| 9/9 [00:00<00:00, 96.93it/s]


Epoch 49: Loss = 3.7968, Accuracy = 88.55%


Epoch 50/50: 100%|██████████| 9/9 [00:00<00:00, 77.75it/s]

Epoch 50: Loss = 3.4449, Accuracy = 91.60%
